In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 数据预处理

In [2]:
import modin as pd
from fastai.tabular import *

In [4]:
root = Path('../test_A')
root
root.ls()

PosixPath('../test_A')

[PosixPath('../test_A/ad_static_feature.out'),
 PosixPath('../test_A/ad_operation.dat'),
 PosixPath('../test_A/test_sample.dat'),
 PosixPath('../test_A/user'),
 PosixPath('../test_A/imps_log')]

In [5]:
user = root/'user'
user.ls()
imps_log = root/'imps_log'
imps_log.ls()

[PosixPath('../test_A/user/user_data')]

[PosixPath('../test_A/imps_log/totalExposureLog.out')]

- 历史曝光日志数据文件

In [6]:
col_names = ['广告请求id', '广告请求时间', '广告位id', '用户id', '曝光广告id', '曝光广告素材尺寸', '曝光广告出价bid',
             '曝光广告pctr', '曝光广告quality_ecpm', '曝光广告totalEcpm']

In [7]:
imps_log = pd.read_csv(imps_log/'totalExposureLog.out', sep='\t', header=None, names=col_names, iterator=True)
imps_log.get_chunk(5)

,广告请求id,广告请求时间,广告位id,用户id,曝光广告id,曝光广告素材尺寸,曝光广告出价bid,曝光广告pctr,曝光广告quality_ecpm,曝光广告totalEcpm
0,53991770,1550409746,94,1160618,451525,50,46,47.217,944.34,3122.340
1,25942318,1550370892,79,203814,214797,64,10,49.094,981.88,1471.880
2,66156247,1550416600,18,808543,92253,40,96,3.824,76.48,443.584
3,5935886,1550365898,198,7270,160082,64,85,6.123,122.46,642.915
4,11624425,1550361159,168,852707,253902,64,60,4.329,86.58,346.320


- 用户特征属性文件

In [8]:
col_names = ['用户id', 'Age', 'Gender', '地域', '婚恋状态', '学历', '消费能力',
             '设备', '工作状态', '连接类型', '行为兴趣']

In [9]:
user_data = pd.read_csv(user/'user_data', sep='\t', header=None, names=col_names, iterator=True)
user_data.get_chunk(5)

,用户id,Age,Gender,地域,婚恋状态,学历,消费能力,设备,工作状态,连接类型,行为兴趣
0,624218,753,2,"523,7589,12527,9019,14605,13489,302,6222,14214...",0,6,1,2,0,4,"32612,34406,3413,33059,4275,4667,4561,34051,42..."
1,271475,601,3,"9019,9032,663,12527,14532,2416,2599,1987,9687,...",13,7,2,2,0,2,"4990,4992,3413,33061,4115,4876,2393,34167,3441..."
2,44439,819,3,"12527,14188,9841,9019,14605,6222,9962,6643,172...",13,7,3,2,4,4,"3099,4403,34414,431,34415,5108,31686,3240,3229..."
3,863717,1,2,"12527,3435,2398,11069,1292,9019,14605,14348,62...",6,5,3,2,0,2,"34293,845,3413,3099,33061,33059,4120,4410,4415..."
4,1305469,714,3,"8928,9019,1540,719,12527,6051,10545,12118,7107...",13,5,3,2,0,2,"32612,34289,34406,3413,4275,4410,4561,34051,34..."


- 广告静态数据

In [10]:
col_names = ['广告id', '创建时间', '广告账户id', '商品id', '商品类型', '广告行业id', '素材尺寸']

In [11]:
ad_static_feature = pd.read_csv(root/'ad_static_feature.out', sep='\t', header=None, names=col_names, low_memory=False, iterator=True)
ad_static_feature.get_chunk(5)

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,106452,1529958950,22226,16088,13,225,NaN
1,233649,1538221936,25681,7356,13,136,1.0
2,547531,1550731020,20696,-1,1,186,40.0
3,707841,1551857857,3968,-1,3,186,40.0
4,457009,1550439402,23614,7447,13,172,NaN


- 广告操作数据

In [12]:
col_names = ['广告id', '创建/修改时间', '操作类型', '修改字段', '操作后的字段值', '人群定向']

In [13]:
ad_operation = pd.read_csv(root/'ad_operation.dat', sep='\t', header=None, names=col_names, iterator=True)
ad_operation.get_chunk(5)

,广告id,创建/修改时间,操作类型,修改字段,操作后的字段值,人群定向
0,593323,0,2,2,90,NaN
1,593323,0,2,3,all,NaN
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065...",NaN
3,593323,20190217000000,1,1,0,NaN
4,593323,20190218233855,1,2,90,NaN


- 测试数据

In [14]:
col_names = ['样本id', '广告id', '创建时间', '素材尺寸', '广告行业id', '商品类型', '商品id', '广告账户id', '投放时段', '人群定向', '出价']

In [15]:
test_sample = pd.read_csv(root/'test_sample.dat', sep='\t', header=None, names=col_names, iterator=True)
test_sample.get_chunk(5)

,样本id,广告id,创建时间,素材尺寸,广告行业id,商品类型,商品id,广告账户id,投放时段,人群定向,出价
0,1,394352,1529648412,34,84,13,29663,26657,"281474976645120,281474976645120,28147497664512...","age:819,608,988,741,202,837,400,394,942,361,72...",120
1,2,585401,1553076190,40,221,1,-1,6262,"281474976579587,281474976579587,28147497657958...","age:819,433,479,741,229,347,522,79,753,601|edu...",42
2,3,419408,1553031394,30,122,13,32110,17436,"17592185782272,17592185782272,17592185782272,1...",all,6
3,4,405326,1553238836,64,136,1,-1,22359,"281474976694272,281474976694272,28147497669427...","age:333,1|gender:2|area:11505,1874,3790,4566,5...",181
4,5,578942,1541191585,34,12,13,6372,24082,"68719214592,68719214592,68719214592,6871921459...","age:819,608,988,741,202,837,400,394,942,361,72...",31


In [20]:
data = test_sample.get_chunk(5)

In [26]:
data.iloc[1, 9]  # 人群定向

'age:819,608,988,741,202,837,400,394,942,361,728,753,601,433,217,844,638,1,366,479,229,393,347,787,340,731,522,333,829,79,739'

In [27]:
data.iloc[1, 8]  # 投放时段

'267386880,267386880,267386880,267386880,267386880,267386880,267386880'

- 分块读取文件(整体读取) chunksize
- 迭代读取，测试数据使用 iterator

In [17]:
test_sample1 = pd.read_csv(root/'test_sample.dat', sep='\t', header=None, names=col_names, chunksize=8192)

In [19]:
for chunk in test_sample1:
    print(chunk.shape)

(8192, 11)
(8192, 11)
(3906, 11)
